In [1]:
from mesa import Model, Agent
from mesa.time import StagedActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from typing import List
import random
import pandas as pd


### Variables that can be adjusted

In [2]:
perception_factor = 0.25 #the factor multiplied with attractiveness difference
match_range = 1 #the range in which people match below their own perceived attractiveness
rand_range = 2 #the variation the match_range can take
evaluation_interval = 20 #the interval between evaluations/how many swipes per "session"

### Agent Class

In [3]:
class DatingProfile(Agent):
    def __init__(self, unique_id, model, woman: bool):
        super().__init__(unique_id, model)
        self.id = unique_id #each agents has a unique ID
        self.mate_value = random.randrange(0, 10) #each agent has random mate value from 0 to 10
        self.p_mate_value = self.mate_value #before interaction perceived mate value is equal to mate_value
        self.woman = woman #each agent is either TRUE or FALSE in regards to being a woman
        self.swipe_list = [] #profiles that have been swiped yes on this round
        self.match_list = [] #profiles that have been matched with
        self.total_swipe_list = [] #swipe list that contains swipes from all rounds 
        self.swipe_queue = [] #list of agents to swipe

        self.model = model
        self.schedule = model.schedule 

    def evaluate_swipe(self, other_agent: "DatingProfile"): 
        """
        Function that determines when to swipe yes or no
        """
        if (other_agent.mate_value >= self.p_mate_value - (match_range * random.randrange(0,rand_range))):
            return True
        else:
            return False
    
    def evaluate_perception(self, other_agent: "DatingProfile", match: bool):
        """
        Function that determines how match outcome affects perceived mate value
        """
        # calculate difference between own perceived mate value and the other persons mate value
        self.att_diff = abs(self.p_mate_value - other_agent.mate_value)
        
        if (match):
            p_change = (self.att_diff)/10 * perception_factor
            if (self.att_diff > 1 and self.p_mate_value + p_change < 10):
                self.p_mate_value += p_change
            
            if (self.att_diff < 1):
                self.p_mate_value = perception_factor/10

            if (self.p_mate_value + p_change > 10):
                self.p_mate_value = 10

        else:
            p_change = (1/self.att_diff)*perception_factor

            if (self.att_diff > 1 and self.p_mate_value - p_change > 0):
                self.p_mate_value -= p_change
            
            if (self.att_diff < 1):
                self.p_mate_value = perception_factor

            if (self.p_mate_value - p_change < 0):
                self.p_mate_value = 0
                
    def register_match_outcome(self, match, agent):
        """
        Function that registers match outcome and adds evaluate_perception function
        """
        if (match == True):
            self.evaluate_perception(agent, True) #Adds (positive) evaluate perception function
            self.match_list.append(agent) #Appends match to match list for first agent

            agent.evaluate_perception(self, True)
            agent.match_list.append(self)
        else:
            self.evaluate_perception(agent, False) #Adds (negative) evaluate perception function   

        agent.swipe_list = [x for x in agent.swipe_list if x["agent"] != self]
        self.swipe_list = [y for y in self.swipe_list if y["agent"] != agent]  

    def swipe_stage(self): 
        """
        First stage of StagedActivation. A function that creates swipe using the evaluate_swipe function made earlier + appends to swipe list
        """

        a = self.swipe_queue[-1]

        if a in self.swipe_list: #if a already exists in swipe list, swipe it of a
            self.swipe_list.remove(a)

        if (self.evaluate_swipe(a) == True): #If conditions are met a swipe is appended to the swipe list
            self.swipe_list.append({'agent': a, 'swipe': True})
            self.total_swipe_list.append(a) #Tracking total number of swipes because swipe_list is cleared for each round

        else:
            self.swipe_list.append({'agent': a, 'swipe': False})


        self.swipe_queue.pop(-1)
                
    def match_check_stage(self): 
        """
        Second stage of StagedActivation. A function that creates match when two people swiped each other 
        """
        if ((self.model.iteration_N / evaluation_interval).is_integer()):
            
            if (len(self.swipe_list) != 0): #Checks if agent has swiped yes to any profiles
                for a in list(self.swipe_list):
                    match_outcome = False #Makes sure that before we compare swipe_list to create matches there is no matches (in case match is TRUE by default)
                    if (len(a["agent"].swipe_list) != 0): #Checks if other agent (a) has swiped yes to anyone
                        for b in list(a["agent"].swipe_list):
                            if (b["agent"] == self and b["swipe"] == True and a["swipe"] == True): #If b is self (original agent that swiped yes to a) a match is made
                                self.register_match_outcome(True, a["agent"])
                                match_outcome = True
                            if (b["agent"] == self and b["swipe"] == False and a["swipe"] == True):
                                self.register_match_outcome(False, a["agent"])
                                match_outcome = True

                    if (match_outcome == False and a["swipe"] == True):
                        self.evaluate_perception(a["agent"], False) #Adds (negative) evaluate perception function 

### Model Class

In [4]:
class DatingModel(Model):
    """
    A model of dating dynamics.
    """
    def __init__(self, N_women, men_factor: int):
        self.men_factor = men_factor #Ratio of men to women
        self.N_women = N_women
        self.N_men = N_women * men_factor
        self.N_agents = N_women + N_women * men_factor #Number of agents in total
        self.iteration_N = 1

        self.agentListW : List[DatingProfile] = [] #Determining that the agents are of the DatingProfile class
        self.agentListM : List[DatingProfile] = []
        self.schedule = StagedActivation(self, ["swipe_stage", "match_check_stage"]) #Telling the model that each step in the simulation contains these two stages

        # Create men
        for i in range(self.N_men):
            m = DatingProfile(i, self, False)
            self.schedule.add(m) #adding the profile to ?
            self.agentListM.append(m)    

        # Create women
        for i in range(self.N_women):
            w = DatingProfile(i + self.N_men, self, True) #Making sure women don't get the same ID as men
            self.schedule.add(w)
            self.agentListW.append(w)
    
    def step(self):
        """
        Advance the model by one step.
        """
        self.iteration_N += 1
        random.shuffle(self.agentListW) #Shuffling the list of men so they don't swipe on the same profiles each round
        random.shuffle(self.agentListM)

        for i in range(len(self.agentListM)):

            m : DatingProfile = self.agentListM[i]
            w : DatingProfile

            if (i > 100):
                w = self.agentListW[i - ((i // 100) * 100)]
            if (i < 100): 
                w = self.agentListW[i]

            m.swipe_queue.append(w) #appending women to men's swipe queue
            w.swipe_queue.append(m) #appending men to women's swipe queue

        self.schedule.step() #activating step (one tick) which is defined by to stages in 'self.schedule = StagedActivation' previously

    def data_collect(self): #A function to collect the data simulated

        self.ID_list = []
        self.woman_list = []
        self.matches_list = []
        self.swipe_list = []
        self.mate_value_list = []
        self.perceived_mate_value_list = []

        #gather a combined list of all the agents in the model
        self.agent_list : List[DatingProfile] = [] 
        self.agent_list.extend(self.agentListM)
        self.agent_list.extend(self.agentListW)

        for i in range(len(self.agent_list)):
            agent = self.agent_list[i]

            self.ID_list.append(agent.id)
            self.woman_list.append(agent.woman)
            self.matches_list.append(len(agent.match_list))
            self.swipe_list.append(len(agent.total_swipe_list))
            self.mate_value_list.append(agent.mate_value)
            self.perceived_mate_value_list.append(agent.p_mate_value)

        data = pd.DataFrame({
            'ID': self.ID_list,
            'Woman': self.woman_list,
            'N_matches': self.matches_list,
            'N_swipes': self.swipe_list,
            'Mate_value:': self.mate_value_list,
            'P_mate_value': self.perceived_mate_value_list
        })

        return data

    def run_model(self, n_steps):
        for i in range(n_steps):
            self.step()

### Running the model 

In [5]:
model = DatingModel(100, 3)
model.run_model(100)

data = model.data_collect()
data.to_csv('dating_data.csv')


AttributeError: 'DatingProfile' object has no attribute 'att_diff'